In [4]:
import pandas as pd
import numpy as np
import re

In [3]:
titanic = pd.read_csv('train.csv')

# 데이터 전처리

    * Name 컬럼에서 호칭 추출(Mr., Mrs. , ... Dr. , Sir, ...) using 정규표현식

In [40]:
titanic.Name.head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [128]:
a = re.compile('Mrs|Mr')

b=[]
for i in titanic.Name.index:
    try:
        c = a.findall(titanic.Name[i])[0]
    except:
        c = np.nan
    
    b.append(c)
titanic['gender']=b

    * abs Mr : 0, Mrs : 1, 나머지 : 2로 변경하여, gender 컬럼을 추가하여 값으로 저장

In [130]:
titanic['gender'].replace({'Mr':0,'Mrs':1,np.nan:2},inplace=True)

    * Age열에 대해 결측값은 해당 데이터의 성별을 토대로 전체 데이터에 대한 성별 값의 평균으로 대체 
    *ex)성별이 남성, age가 결측값인 경우에는 남성 age 평균값으로 대체

In [146]:
titanic.Age.mean()

23.79929292929293

In [164]:
a=titanic[titanic.gender==0].replace({'Age':0},{'Age':titanic[titanic.gender==0].Age.mean()})
b=titanic[titanic.gender==1].replace({'Age':0},{'Age':titanic[titanic.gender==1].Age.mean()})
c=titanic[titanic.gender==2].replace({'Age':0},{'Age':titanic[titanic.gender==2].Age.mean()})

titanic = pd.concat([a,b,c])

In [169]:
titanic= titanic.sort_index()
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
gender         891 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 97.5+ KB


    *FamilySize 컬럼 추가, 컬럼값은 SibSp+Parch값으로 함

In [172]:
titanic['FamilySize']=titanic.SibSp+titanic.Parch

    * Fare열에 대한 평균, 중위수를 구함(극단치 존재 여부 확인)

In [174]:
titanic.Fare.describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

    * Age열에 대해 중복된 데이터는 모두 제거(저장x)

In [180]:
titanic.Age.drop_duplicates(keep=False)

33     66.00
111    14.50
116    70.50
125    12.00
148    36.50
152    55.50
227    20.50
296    23.50
305     0.92
571    53.00
630    80.00
676    24.50
755     0.67
803     0.42
843    34.50
851    74.00
Name: Age, dtype: float64

    * Cabin열에 대해 unique한 데이터를 추출

In [181]:
titanic.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

    * Age열에 대해 카테고리형(범주) 데이터로 변경 후 연령대별 생존율/사망율 구하기

In [232]:
titanic.Age.value_counts(bins=[0,20,40,60,titanic.Age.max()])

(20.0, 40.0]      521
(-0.001, 20.0]    220
(40.0, 60.0]      128
(60.0, 80.0]       22
Name: Age, dtype: int64

In [209]:
cut = pd.cut(titanic.Age,bins=[0,20,40,60,titanic.Age.max()])
pd.value_counts(cut)

(20.0, 40.0]    521
(0.0, 20.0]     220
(40.0, 60.0]    128
(60.0, 80.0]     22
Name: Age, dtype: int64

In [210]:
titanic['age_range']=cut

In [266]:
for i in range(len(titanic.age_range.unique())):
    a = titanic[titanic['age_range']==titanic.age_range.unique()[i]]
    a = a.Survived.value_counts(1)

    print(f'{titanic.age_range.unique()[i]} 생존율:{a[1]}, 사망율:{a[0]}')

(20.0, 40.0] 생존율:0.3474088291746641, 사망율:0.6525911708253359
(40.0, 60.0] 생존율:0.390625, 사망율:0.609375
(0.0, 20.0] 생존율:0.4818181818181818, 사망율:0.5181818181818182
(60.0, 80.0] 생존율:0.22727272727272727, 사망율:0.7727272727272727


    * Embarked 컬럼값에 대해 항구별 승선인원 세기(비율)

In [272]:
titanic.Embarked.value_counts(1)

S    0.724409
C    0.188976
Q    0.086614
Name: Embarked, dtype: float64

    * Survived컬럼에 대해 생존(1)/사망(0) 인원수 및 비율

In [274]:
print(titanic.Survived.value_counts())
print(titanic.Survived.value_counts(1))

0    549
1    342
Name: Survived, dtype: int64
0    0.616162
1    0.383838
Name: Survived, dtype: float64


    * 성별에 따른 생존율/사망율 구하기

In [290]:
ss = titanic[['Sex','Survived']]
print(ss[ss.Sex=='male'].Survived.value_counts(1))
print(ss[ss.Sex=='female'].Survived.value_counts(1))

0    0.811092
1    0.188908
Name: Survived, dtype: float64
1    0.742038
0    0.257962
Name: Survived, dtype: float64
